In [1]:
import cv2
import ultralytics
from ultralytics import YOLO
import numpy as np
from collections import defaultdict
import time

In [2]:
# class ids

detect = YOLO("../Model/FineTune/runs/detect/train17/weights/best.pt", task='track')

classes = detect.names

class_dict = {v:k for k,v in classes.items()}


hand_id   = class_dict["hand"]
shelf_id  = class_dict["shelf"]
person_id = class_dict["person"]

object_ids    = {
    class_dict["Vaseline"],
    class_dict["glow and lovely"],
    class_dict["haircream"],
    class_dict["hygienix sanitizer"],
    class_dict["nyka bb cream"],
    class_dict["parachute coconut oil"],
    class_dict["perfume"],
    class_dict["plum"],
}


def iou(boxA, boxB):

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    if inter == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return inter / float(boxAArea + boxBArea - inter)

In [3]:
detect = YOLO("../Model/FineTune/runs/detect/train17/weights/best.pt")

#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("../live_check/vid_1.mp4")
cv2.namedWindow('AutoCart', cv2.WINDOW_NORMAL)

picked        = defaultdict(int)
putback       = defaultdict(int)
pending       = defaultdict(int) 
Final_Cart    = {}
object_state  = {}

last_shelf = []                                      

def center_true(px, py, box):
    return box[0] <= px <= box[2] and box[1] <= py <= box[3]

while True:
    a, frames = cap.read()
    if not a:
        break

    frames = cv2.rotate(frames, cv2.ROTATE_90_CLOCKWISE)

    result = detect.track(source=frames, show=False,conf=0.35, max_det=25,nms=True, iou=0.4, agnostic_nms=False,imgsz=640, augment=False,persist=True, 
                          verbose=False,tracker="botsort.yaml")
                        
                            

    current_time = time.time()

    # getting the boxes min and max lenghts from the screen.

    total_detected = result[0].boxes           
    shelf = [i.xyxy[0].cpu().numpy().tolist() for i in total_detected if int(i.cls[0]) == shelf_id]
    hand  = [i.xyxy[0].cpu().numpy().tolist() for i in total_detected if int(i.cls[0]) == hand_id]


    # making sure that shelf doesn't go away 
    if shelf:
        last_shelf = shelf
    else:
        shelf = last_shelf

    for obj in total_detected:
        cls_id = int(obj.cls[0])
        if cls_id not in object_ids:
            continue  # skipping the unwanted objects


        # current box edges
        xmin, ymin, xmax, ymax = obj.xyxy[0].cpu().numpy().tolist()
        cx, cy = (xmin + xmax) / 2, (ymin + ymax) / 2 # center of the object

        within_shelf = any(center_true(cx, cy, s) for s in shelf) # if the object is within the shelf or no
        touch = any(iou([xmin, ymin, xmax, ymax], h) > 0.01 for h in hand) # intersection for hand and object

        # dummy track
        prev = object_state.get(cls_id, {'in_shelf': within_shelf,
                                          'picked_time': None,
                                          'putback_time' : None,
                                          'last_touch': None})

        if touch:
            prev['last_touch'] = current_time


        recent_touch = prev['last_touch'] and (current_time - prev['last_touch'] <= 1.0)

        # was in shelf before but not now
        try:
            if prev['in_shelf'] and not within_shelf and recent_touch and pending[cls_id]==0:
                picked[cls_id] += 1
                pending[cls_id] += 1          
                prev['picked_time'] = time.time()
                prev['last_touch']  = None
        except:
            if prev['in_shelf'] and not within_shelf and recent_touch:
                picked[cls_id] += 1
                pending[cls_id] += 1          
                prev['picked_time'] = time.time()
                prev['last_touch']  = None

        # was not in shelf before but present now  
        if within_shelf and recent_touch and pending[cls_id] > 0:
            putback[cls_id] += 1
            pending[cls_id] -= 1          
            prev['putback_time'] = time.time()
            prev['last_touch']  = None

        prev['in_shelf'] = within_shelf
        object_state[cls_id] = prev

    annoted = result[0].plot()
    cv2.imshow('AutoCart', annoted)

    if cv2.waitKey(1) & 0xFF == ord('q'):

        break

cap.release()
cv2.destroyAllWindows()


if len(picked)>0:
    for pick,count in picked.items():
        Final_Cart[classes[pick]] = Final_Cart.get(classes[pick],0)+count
        print(f'Picked {count}-{classes[pick]}')

if len(putback)>0:
    print()
    for put,count in putback.items():
        Final_Cart[classes[put]] = Final_Cart.get(classes[put],0)-count
        print(f'Returned {count}-{classes[put]}')
print()
for item,count in pending.items():
    if count>0:
        print(f'CartValue is {count}-{classes[item]}')
    else:
        print('Nothing Bought, Cart is Empty')

Picked 1-parachute coconut oil
Picked 1-nyka bb cream

Returned 1-parachute coconut oil

Nothing Bought, Cart is Empty
Nothing Bought, Cart is Empty
Nothing Bought, Cart is Empty
CartValue is 1-nyka bb cream
